### Random Forest Classifier WandB Sweep

In [1]:
import pandas as pd

In [2]:
idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')

In [7]:
from sklearn.preprocessing import StandardScaler

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

#PredictorScaler=StandardScaler()
#PredictorScalerFit=PredictorScaler.fit(X)
#X=PredictorScalerFit.transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
### shuffle/inorder train and test sets:
# , shuffle=False
# , random_state=42

#### ggf irgendwann ts_cv zur cross validation

from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=4,max_train_size=10000)

all_splits = list(ts_cv.split(X_train, y_train))
train_0, test_0 = all_splits[0]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=8)
ts_cv

In [8]:
sweep_configuration = {
    "project": "RandomForestClass",
    "name": "RFC-sweep-split-inorder-not-scaled",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [8, 16, 32, 64, 128, 256, 512]
        },
        "criterion": {
            "values": ['entropy','gini']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        }
    }
}

In [9]:
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import wandb
import wandb

def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train.ravel())
    score_validation = model.score(X_test, y_test.ravel())

    acc = accuracy_score(y_test.ravel(), y_pred.ravel())
    loss = zero_one_loss(y_test.ravel(), y_pred.ravel())

    wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"loss": loss})
    wandb.log({"score_training":score_training, "score_validation":score_validation})


In [12]:
def my_train_func_cv():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    cv_res = cross_val_score(model,X_train,y_train,cv = ts_cv, scoring='accuracy')

    #score_training = model.score(X_train, y_train)
    #score_validation = model.score(X_test, y_test)

    dict = {"acc1": cv_res[0],
            "acc2": cv_res[1],
            "acc3": cv_res[2],
            "acc4": cv_res[3],
            "acc5": cv_res[4],
            "acc6": cv_res[5],
            "acc7": cv_res[6],
            "acc8": cv_res[7]
            }

    wandb.log({"cv_accuracy": dict})
    #wandb.log({"score_training":score_training,"score_validation":score_validation})


In [11]:
# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration, project="RandomForestClassifier")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)


Create sweep with ID: 1wpio7zz
Sweep URL: https://wandb.ai/jonathanweske/RandomForestClassifier/sweeps/1wpio7zz


wandb: Agent Starting Run: wwvi51du with config:
wandb: 	bootstrap: False
wandb: 	criterion: gini
wandb: 	max_depth: 5
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 8
wandb: 	min_samples_split: 4
wandb: 	n_estimators: 32
wandb: Currently logged in as: jonathanweske (use `wandb login --relogin` to force relogin)


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.46593
loss,0.53407
score_training,0.50354
score_validation,0.46593


wandb: Agent Starting Run: 62uiemvp with config:
wandb: 	bootstrap: True
wandb: 	criterion: entropy
wandb: 	max_depth: 120
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 6
wandb: 	min_samples_split: 11
wandb: 	n_estimators: 128


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.52169
loss,0.47831
score_training,0.81932
score_validation,0.52169


...

accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.52036
loss,0.47964
score_training,0.64816
score_validation,0.52036


wandb: Agent Starting Run: 7w5s8c5c with config:
wandb: 	bootstrap: False
wandb: 	criterion: entropy
wandb: 	max_depth: None
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 25
wandb: 	min_samples_split: 9
wandb: 	n_estimators: 64


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.52463
loss,0.47537
score_training,0.68689
score_validation,0.52463


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: w7m47ry2 with config:
wandb: 	bootstrap: False
wandb: 	criterion: entropy
wandb: 	max_depth: 6
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 7
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 64


wandb: Ctrl + C detected. Stopping sweep.
